In [1]:
import os
import json
import numpy as np

In [28]:
with open(r'J:\AI based welding detection in Gas metal arc welding\weldingpool data\15.json', 'r', encoding='utf-8') as f:
    labelme = json.load(f)

In [4]:
labelme.keys()

dict_keys(['version', 'flags', 'shapes', 'imagePath', 'imageData', 'imageHeight', 'imageWidth'])

In [5]:
labelme['version']

'5.3.1'

In [6]:
labelme['flags']

{}

In [7]:
labelme['imagePath']

'15.png'

In [8]:
labelme['imageData']

In [9]:
labelme['imageHeight']

1080

In [10]:
labelme['imageWidth']

1440

In [11]:
labelme['shapes']

[{'label': 'leftpoint',
  'points': [[680.0, 448.3333333333333]],
  'group_id': None,
  'description': '',
  'shape_type': 'point',
  'flags': {}},
 {'label': 'rightpoint',
  'points': [[896.1111111111111, 442.77777777777777]],
  'group_id': None,
  'description': '',
  'shape_type': 'point',
  'flags': {}},
 {'label': 'highpoint',
  'points': [[793.3333333333333, 385.55555555555554]],
  'group_id': None,
  'description': '',
  'shape_type': 'point',
  'flags': {}},
 {'label': 'lowpoint',
  'points': [[794.4444444444445, 500.0]],
  'group_id': None,
  'description': '',
  'shape_type': 'point',
  'flags': {}},
 {'label': 'weldingpool',
  'points': [[668.8888888888889, 345.0],
   [913.3333333333333, 523.3333333333334]],
  'group_id': None,
  'description': '',
  'shape_type': 'rectangle',
  'flags': {}}]

In [12]:
coco = {}

In [13]:
coco['info'] = {}
coco['info']['description'] = 'Labelme2coco keypoint format script '
coco['info']['year'] = 2023
coco['info']['date_created'] = '2023/12/12'

In [14]:
class_list= {
    'supercategory': 'weldingpool',
    'id': 1,
    'name': 'weldingpool',
    'keypoints': ['leftpoint', 'rightpoint', 'highpoint', 'lowpoint'], 
}

In [15]:
coco['categories'] = []
coco['categories'].append(class_list)

coco['images'] = []
coco['annotations'] = []

IMG_ID = 0
ANN_ID = 0

In [16]:
def process_single_json(labelme, image_id=1):
    '''
    输入labelme的json数据，输出coco格式的每个框的关键点标注信息
    '''
    
    global ANN_ID
    
    coco_annotations = []
    
    for each_ann in labelme['shapes']: 

        if each_ann['shape_type'] == 'rectangle': 

           
            bbox_dict = {}
            bbox_dict['category_id'] = 1
            bbox_dict['segmentation'] = []
            
            bbox_dict['iscrowd'] = 0
            bbox_dict['segmentation'] = []
            bbox_dict['image_id'] = image_id
            bbox_dict['id'] = ANN_ID
            # print(ANN_ID)
            ANN_ID += 1

            
            bbox_left_top_x = min(int(each_ann['points'][0][0]), int(each_ann['points'][1][0]))
            bbox_left_top_y = min(int(each_ann['points'][0][1]), int(each_ann['points'][1][1]))
            bbox_right_bottom_x = max(int(each_ann['points'][0][0]), int(each_ann['points'][1][0]))
            bbox_right_bottom_y = max(int(each_ann['points'][0][1]), int(each_ann['points'][1][1]))
            bbox_w = bbox_right_bottom_x - bbox_left_top_x
            bbox_h = bbox_right_bottom_y - bbox_left_top_y
            bbox_dict['bbox'] = [bbox_left_top_x, bbox_left_top_y, bbox_w, bbox_h] # 左上角x、y、框的w、h
            bbox_dict['area'] = bbox_w * bbox_h
            
            
            for each_ann in labelme['shapes']: 
                if each_ann['shape_type'] == 'polygon': 
                    
                    first_x = each_ann['points'][0][0]
                    first_y = each_ann['points'][0][1]
                    if (first_x>bbox_left_top_x) & (first_x<bbox_right_bottom_x) & (first_y<bbox_right_bottom_y) & (first_y>bbox_left_top_y): # 筛选出在该个体框中的关键点
                        bbox_dict['segmentation'] = list(map(lambda x: list(map(lambda y: round(y, 2), x)), each_ann['points'])) # 坐标保留两位小数
                        # bbox_dict['segmentation'] = each_ann['points']

            
            bbox_keypoints_dict = {}
            for each_ann in labelme['shapes']: 
                
                if each_ann['shape_type'] == 'point': 
                    
                    x = int(each_ann['points'][0][0])
                    y = int(each_ann['points'][0][1])
                    label = each_ann['label']
                    if (x>bbox_left_top_x) & (x<bbox_right_bottom_x) & (y<bbox_right_bottom_y) & (y>bbox_left_top_y): # 筛选出在该个体框中的关键点
                        bbox_keypoints_dict[label] = [x, y]
                        
            bbox_dict['num_keypoints'] = len(bbox_keypoints_dict)
            # print(bbox_keypoints_dict)

            
            bbox_dict['keypoints'] = []
            for each_class in class_list['keypoints']:
                if each_class in bbox_keypoints_dict:
                    bbox_dict['keypoints'].append(bbox_keypoints_dict[each_class][0])
                    bbox_dict['keypoints'].append(bbox_keypoints_dict[each_class][1])
                    bbox_dict['keypoints'].append(2) 
                else: 
                    bbox_dict['keypoints'].append(0)
                    bbox_dict['keypoints'].append(0)
                    bbox_dict['keypoints'].append(0)
                    
            coco_annotations.append(bbox_dict)
            
    return coco_annotations

In [17]:
labelme_json_path = r'J:\AI based welding detection in Gas metal arc welding\weldingpool data\15.json'

with open(labelme_json_path, 'r', encoding='utf-8') as f:
    labelme = json.load(f)

In [18]:
process_single_json(labelme)

[{'category_id': 1,
  'segmentation': [],
  'iscrowd': 0,
  'image_id': 1,
  'id': 0,
  'bbox': [668, 345, 245, 178],
  'area': 43610,
  'num_keypoints': 4,
  'keypoints': [680, 448, 2, 896, 442, 2, 793, 385, 2, 794, 500, 2]}]

In [19]:
IMG_ID = 0
ANN_ID = 0

In [27]:
for labelme_json in os.listdir(): 
    
    if labelme_json.split('.')[-1] == 'json':
        
        with open(labelme_json, 'r', encoding='utf-8') as f:
            
            labelme = json.load(f)
            
            
            img_dict = {}
            img_dict['file_name'] = labelme['imagePath']
            img_dict['height'] = labelme['imageHeight']
            img_dict['width'] = labelme['imageWidth']
            img_dict['id'] = IMG_ID
            coco['images'].append(img_dict)
            
           
            coco_annotations = process_single_json(labelme, image_id=IMG_ID)
            coco['annotations'] += coco_annotations
            
            IMG_ID += 1
            
            print(labelme_json, '已处理完毕')

    else:
        pass

In [21]:
if not os.path.exists('output_coco'):
    os.mkdir('output_coco')
    print('创建新目录 output_coco')

创建新目录 output_coco


In [22]:
coco_path = 'output_coco/coco_sample.json'

In [23]:
with open(coco_path, 'w') as f:
    json.dump(coco, f, indent=2)

In [24]:
coco

{'info': {'description': 'Labelme2coco keypoint format script ',
  'year': 2023,
  'date_created': '2023/12/12'},
 'categories': [{'supercategory': 'weldingpool',
   'id': 1,
   'name': 'weldingpool',
   'keypoints': ['leftpoint', 'rightpoint', 'highpoint', 'lowpoint']}],
 'images': [],
 'annotations': []}

In [25]:
!pip install pycocotools

  Obtaining dependency information for pycocotools from https://files.pythonhosted.org/packages/24/b2/ef28a34cf6ca50b6b2f7ad81e5837ed45c252ffef22f5a704b94141ea842/pycocotools-2.0.7-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/85.8 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/85.8 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/85.8 kB ? eta -:--:--
   -------------- ------------------------- 30.7/85.8 kB 262.6 kB/s eta 0:00:01
   --------------------------------- ------ 71.7/85.8 kB 491.5 kB/s eta 0:00:01
   ---------------------------------------- 85.8/85.8 kB 484.2 kB/s eta 0:00:00


In [26]:
from pycocotools.coco import COCO

my_coco = COCO(coco_path)

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
